In [1]:
path='G:/Shared drives/BeStarsMiMeS/UpdatedFiles/' #Patrick's google file stream path

import LSDpy.lsdpy
import specpolFlow as pol


import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import itertools

import pyRaven as rav

loading specpolFlow package


In [2]:
#Accessing the google sheets

sheet_id = '1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk'
sheet_name = 'Stars'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
StarData=pd.read_csv(url)

sheet_name = 'Observations'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
Observations=pd.read_csv(url)

In [3]:
StarData

,HD,Name,Nobs,T,log(g),ModelCode,Adopted-vsini,Halpha shape,Notes,BadvsiniFlag,...,pyRavenFlag,AsifgoodFlag,TweakgoodFlag,i_min [deg],Unnamed: 19,Unnamed: 20,hybrid0.02 chi2,norm0.02 chi2,Unnamed: 23,Unnamed: 24
0,6226.0,hd6226,1,15000,4.0,T15000g40,120.00,absorption,"no disk during observation, rotational modulat...",1.0,...,NaN,NaN,NaN,10.369760,NaN,NaN,NaN,NaN,NaN,NaN
1,7636.0,hd7636,2,15000,4.0,T15000g40,220.00,asymetric single,binary? no tess observation,NaN,...,NaN,NaN,NaN,17.457603,NaN,NaN,NaN,NaN,NaN,NaN
2,10144.0,hd10144,1,20000,3.5,T20000g35,245.31,bowl,binary or pulsator,NaN,...,1.0,1.0,1.0,29.340582,NaN,NaN,NaN,NaN,NaN,NaN
3,10516.0,hd10516,1,25000,4.0,T25000g40,382.04,asymetric double,sdO companion https://iopscience.iop.org/artic...,1.0,...,1.0,1.0,1.0,34.055798,NaN,NaN,done,NaN,NaN,NaN
4,11415.0,hd11415,1,18000,4.0,T18000g40,48.00,absorption,not a binary https://iopscience.iop.org/articl...,1.0,...,NaN,NaN,NaN,6.892103,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,217891.0,hd217891,5,15000,3.0,T15000g30,95.00,single,not a binary https://iopscience.iop.org/articl...,1.0,...,1.0,1.0,1.0,13.886540,jon,NaN,NaN,NaN,NaN,NaN
74,221507.0,hd221507,1,10500,4.0,T10500g40,40.00,absorption,Not a Be star,1.0,...,NaN,NaN,NaN,4.588566,NaN,NaN,NaN,NaN,NaN,NaN
75,224686.0,hd224686,2,13000,4.0,T13000g40,286.16,shell,NaN,1.0,...,1.0,1.0,NaN,35.450543,NaN,NaN,NaN,NaN,NaN,NaN
76,239712.0,hd239712,1,21000,4.0,T21000g40,304.86,asymetric double,NaN,NaN,...,1.0,1.0,NaN,35.450543,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
star='hd10144'

param
fname='{}05-RavenSetup/hybrid_maskdepth0.02_tellClean/{}.h5'.format(path,star)
param=rav.params.read_parameters('{}05-RavenSetup/params/{}.json'.format(path,star))
datapacket=rav.data.read_packet(fname)

out=rav.loop_speed.loop_speed(param,datapacket,path='{}06-Raven-calculations/test/'.format(path)) #could take a while depending on how fast your computer is

Using 10961.674308651056 grid point on the surface
Max velocity needed: 34.90418577162764 vdop
Number of wavelength/velocity grid points: 7201
Starting inclination loop 0/31
5486.940589427948
Starting inclination loop 1/31
3730.9535279273987
Starting inclination loop 2/31
3890.3561537265778
Starting inclination loop 3/31
15480.112622499466
Starting inclination loop 4/31
3757.1607217788696
Starting inclination loop 5/31
3843.887179851532
Starting inclination loop 6/31
17196.487867116928
Starting inclination loop 7/31
4007.6828253269196
Starting inclination loop 8/31
4002.1578946113586
Starting inclination loop 9/31
4004.389035463333
Starting inclination loop 10/31
4038.3269209861755
Starting inclination loop 11/31
3829.5951187610626
Starting inclination loop 12/31
3794.1975090503693
Starting inclination loop 13/31
5129.224828958511
Starting inclination loop 14/31
5047.778907775879
Starting inclination loop 15/31
11745.696862220764
Starting inclination loop 16/31
4175.534140825272
Starti

In [6]:
star='hd170783'
fname='{}05-RavenSetup/hybrid_maskdepth0.02_tellClean/{}.h5'.format(path,star)
param=rav.params.read_parameters('{}05-RavenSetup/params/{}.json'.format(path,star))
datapacket=rav.data.read_packet(fname)
folder='{}06-RavenCalculations/data/{}'.format(path,star)
output='test'.format(path)

# Calculate the likelihoof for the odds ratio calculations
rav.BayesObjects.create_lnLH_odds_from_chi(folder, param, datapacket,output)

# Calculate the likelihood for the parameter estimation
# For this you will need a grid for the noise scale parameter
param['grid']['noise_grid'] = np.arange(0.1,2.1,0.1)
rav.BayesObjects.create_lnLH_pars_from_chi(folder, param, datapacket,output)

# Calculate all of the posterior probabilities
rav.BayesObjects.combine_obs(datapacket.nobs,output)

# Make some diagnostic graphs
rav.BayesObjects.overview_plots(datapacket.nobs,output)

In [17]:
print('test')
!dir
print('test')
!cd 05-RavenSetup

test
 Volume in drive C is Windows-SSD
 Volume Serial Number is F2CE-9A03

 Directory of c:\Users\pjsta\GitHub\BeStarsMiMeS

08/08/2023  07:11 AM    <DIR>          .
08/29/2023  10:41 AM    <DIR>          ..
08/08/2023  07:11 AM            10,244 .DS_Store
08/08/2023  07:11 AM             4,774 00-InputInformation.md
08/08/2023  07:11 AM    <DIR>          00-InputMaterial
08/08/2023  07:11 AM    <DIR>          01-Synth-calculations
08/08/2023  07:11 AM             6,184 01-SyntheticProfileCalculation.md
08/08/2023  07:11 AM             4,708 02-LSDMaskCalculation.md
08/08/2023  07:11 AM    <DIR>          02-Mask-calculations
08/08/2023  07:11 AM    <DIR>          03-LSD-calculations
08/08/2023  07:11 AM             2,577 03-LSDProfileCalculation.md
08/08/2023  07:11 AM    <DIR>          04-Bz-calculations
08/08/2023  07:11 AM             1,363 04-LongitudinalFieldCalculation.md
09/01/2023  12:06 PM    <DIR>          05-RavenSetup
08/08/2023  07:11 AM             3,021 05-RavenSetup.md
